This notebook is part the of Dr. Christoforos Christoforou's course materials. You may not, nor may you knowingly allow others to reproduce or distribute lecture notes, course materials or any of their derivatives without the instructor's express written consent.

# Problem Set 07 - Basics of Neural Network Basics

**Professor:** Dr. Christoforos Christoforou 

In this problem set, you will practice building a basic neural network based on the theory covered and in course and using TensorFlow, an open-source library to help you develop and train ML models. For this problem set, you will need the following library references, which are pre-installed with the colab environment.


In [2]:
import tensorflow.keras 
from tensorflow.keras.callbacks import EarlyStopping 
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.datasets import mnist 
from tensorflow.keras import backend

%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd

## Exercise 1: Load and proprocess the dataset.

For this example, you will use the MNIST dataset, one of the benchmark image dataset. The dataset comprise images of handwritten digits. It is available in the `tensorflow.keras.dataset` module and can be loaded using the `mnist.load_data()` method. The code below showcases how to load the MINST dataset

```python
# Load the image dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
```
The `X_train` has dimensions (60000, 28,28) representing 60000 images, each 28 by 28 pixels. The first axis corresponds the #images and the rest of the dimensions specify the color intensity for each pixel. Similarly, `X_test` has dimension (10000, 28,28) representing 10000 observations.

The `y_train` has dimensions (60000,) with values 0 to 9, representing each of the digit classes.

**Task 1.1** In the cell below load the MNIST Dataset.


In [6]:
# Load the mist dataset 
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Once you load the dataset, you need to convert the images into a format appropriate for the neural network to operate on them. The pre-processing steps are as follow:

* Reshape data array to `channels_last` (i.e. (obs,row,cols,color)) or `channel_first` (i.e. (obs,color,row,cols)) format; depending on what format is supported by the library. Since, tensorflow expects the input vector be be given in a `channel_last` format we need to convert each image to a (obs, row, cols, color) format.
* Convert data array to float32.
* Normalize data array (i.e. divide by 255)
* Convert labels to categorical variables (i.e. one-hot encoding) using the tensorflow.keras.utils.to_categorical(y_train).

The code below showcases one why this can be done in python, for a `ndarray` `X_train` , and how to encode a label vector to one-hot encoding.

```python
# Get the shape of the ndarray
(Nobs_train, n_rows, n_cols) = X_train.shape

# Reshpep array to (obs,row,cols,color); convert to float and normalize
X_train = X_train.reshape(Nobs_train,n_rows,n_cols,1).astype('float32')/255

# Convert y_train labels to one-hot encoding
y_train = tensorflow.keras.utils.to_categorical(y_train)
```
**Task 1.2** In the cell below, standardize the X_train, X_test, y_train and y_test, by applying the necessary preprocessing steps outlined above.

In [7]:
# Implement task 1.2 in this cell
# Get the shape of the ndarray

(Nobs_train, n_rows, n_cols) = X_train.shape

# Reshpep array to (obs,row,cols,color); convert to float and normalize
X_train = X_train.reshape(Nobs_train,n_rows,n_cols,1).astype('float32')/255

# Convert y_train labels to one-hot encoding
y_train = tensorflow.keras.utils.to_categorical(y_train)

## Exercise 2:Building a Neural Network using high-level Keras API

In this introductory example, we will build a neural network using high-level Keras API. For that, you will need several objects provided by the API; which include the `Sequential` model which is defined under the module `tensorflow.keras.models`, and two computational layers, namely the `Flatten` layer and the `Dense` layer, which are defined under the `tensorflow.keras.layers module`. These libraries have been imported at the beginning of the notebook, but to make this section self-contained, we re-import them in the cell below.

In [2]:
import tensorflow.keras 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten
from keras.datasets import mnist
import numpy as np
import time
# Load the mist dataset 
(X_train, y_train), (X_test, y_test) = mnist.load_data()

**Defining the neural network architecture**
We can define the neural network architecture using the keras API. First, we define a general `Sequential` model and then `add` to it the various computational layer. For our example, we first need to add a `Flatten` layer to convert the 2D image into a 1D vector, and then `add` two dense layers, one layer has 256 nodes; and the second one - which serves as an output layer- has 10 nodes. Each element in the output layer represents one of the categories we are trying to classify (i.e. digits 0 to 9) and its value correspond to the probability the input belong to each class. We can define this architecture using the `Keras API` as follows:

```python
model = Sequential()
model.add(Flatten()) # add a layer to convert the 2D image to a 1D vector
model.add(Dense(256,activation='relu'))
model.add(Dense(10, activation='softmax'))
```
Notice, the first `Dense` layer uses the `relu` activation function, and the output `Dense` layer uses the `softmax` activation function. Recall, the softmax normalizes the output vectors so that all entries are positive and sum up to one; thus the resulting output can be thought of as a probability over the categories.

**Task 2.1** In the cell below, define the neural network architecture specified in the section above.

In [4]:
# Implement task 2.1 
import tensorflow.keras 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten
from keras.datasets import mnist
import numpy as np
import time
# Load the mist dataset 
(X_train, y_train), (X_test, y_test) = mnist.load_data()

model = Sequential()
model.add(Flatten()) # add a layer to convert the 2D image to a 1D vector
model.add(Dense(256,activation='relu'))
model.add(Dense(10, activation='softmax'))

**Compile the Neural Network Architecture**
Once we define the model, we need to compile it using the `model.compile` method of the API. As part of the compiling, we define can define the following parameters (among others):.

* the **loss** function* to use for training the model.
* the **optimizer** method to use in optimizing and, typically, we use the `adam` optimizer;
* any *metrics* we want to track while fitting the model (i.e. accuracy)

In this example, since we are dealing with a multi-class classification, we can set the loss function to be `categorical_crossentropy`. For your reference, the categorical cross entropy (CCE) is calculated as:

$$
CCE = -\sum_{c\in C} y_c log(\sigma(\hat{y})_c)
$$

where $y_c$ is the ground truth value of the c-th element in the output vector (i.e. c-th class), $\hat{y}_c$ is the c-th element in the predicted output vector, and $\sigma(.)$ is the `softmax` method. Notice, that the categorical cross entropy implementation in TensorFlow expects the label vector $y$ to be given as a `one-hot-encoding` form.

The code below illustrates how to apply the `model.compile` method, to compile the network architecture

```python
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

```

**Task 2.2:** Compile the Neural Network Architecture you defined in task 2.1



In [5]:
# Use this cell to implement task 2.2 
# Implement task 2.1 
import tensorflow.keras 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten
from keras.datasets import mnist
import numpy as np
import time
# Load the mist dataset 
(X_train, y_train), (X_test, y_test) = mnist.load_data()

model = Sequential()
model.add(Flatten()) # add a layer to convert the 2D image to a 1D vector
model.add(Dense(256,activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

**Training the model**

Once we defined and compiled our model, we can train it by calling the `model.fit` method of the API. The `model.fit` expects as input  the training set and a number of fitting configuration parameters. Two commonly used parameters are that of `batch_size` and `epochs`. The code below illustrates how to fit a model. 

```python
  # Batch size: refers to the size of data we use in a single pass; 
  batch_size = 128

  # Epochs, the number of times to iterate over the dataset  
  epochs = 15 

  model.fit(X_train,y_train, batch_size=batch_size, epochs = epochs,validation_data=(X_test,y_test))
```
**Task 2.3** Train the model by calling the `model.fit` method


In [34]:
# Use this cell to implement task 2.2 
# Implement task 2.1 
import tensorflow.keras 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten
from keras.datasets import mnist
import numpy as np
import time
# Load the mist dataset 
(X_train, y_train), (X_test, y_test) = mnist.load_data()

model = Sequential()
model.add(Flatten()) # add a layer to convert the 2D image to a 1D vector
model.add(Dense(256,activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(), metrics='accuracy')

# Use this cell in implement task 2.3 
# Batch size: refers to the size of data we use in a single pass; 
batch_size = 128

  # Epochs, the number of times to iterate over the dataset  
epochs = 15 

model.fit(X_train,y_train, batch_size=batch_size, epochs = epochs,validation_data=(X_test,y_test))

Epoch 1/15
469/469 [==============================] - 1s 3ms/step - loss: 4.0040 - accuracy: 0.8949 - val_loss: 0.9578 - val_accuracy: 0.9334
Epoch 2/15
469/469 [==============================] - 2s 3ms/step - loss: 0.5886 - accuracy: 0.9446 - val_loss: 0.6541 - val_accuracy: 0.9276
Epoch 3/15
469/469 [==============================] - 2s 3ms/step - loss: 0.2986 - accuracy: 0.9578 - val_loss: 0.5278 - val_accuracy: 0.9458
Epoch 4/15
469/469 [==============================] - 2s 3ms/step - loss: 0.2089 - accuracy: 0.9664 - val_loss: 0.3925 - val_accuracy: 0.9518
Epoch 5/15
469/469 [==============================] - 2s 3ms/step - loss: 0.1737 - accuracy: 0.9688 - val_loss: 0.3719 - val_accuracy: 0.9549
Epoch 6/15
469/469 [==============================] - 1s 3ms/step - loss: 0.1479 - accuracy: 0.9723 - val_loss: 0.3913 - val_accuracy: 0.9514
Epoch 7/15
469/469 [==============================] - 1s 3ms/step - loss: 0.1180 - accuracy: 0.9763 - val_loss: 0.3483 - val_accuracy: 0.9596
Epoch 

**Evaluating the Network model performance**

Once the model has been trained, we can evaluate its performance on a test set by using `model.evaluate` method. The `evaluate` method task as input a validation set and returns a tuple which includes the loss score and an accuracy score. The code below illustrates how to use the evaluate method

```python
(loss_score, accuracy_score) = model.evaluate(X_test,y_test,verbose=0)
```

**Task 2.4:** Evaluate the model you trained in task 2.3 and report its accuracy and loss.


In [52]:
# Use this cell to implement task 2.2 
# Implement task 2.1 
import tensorflow.keras 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten
from keras.datasets import mnist
import numpy as np
import time
# Load the mist dataset 
(X_train, y_train), (X_test, y_test) = mnist.load_data()

model = Sequential()
model.add(Flatten()) # add a layer to convert the 2D image to a 1D vector
model.add(Dense(256,activation='relu'))
model.add(Dense(10, activation='softmax'))
model.add(Dropout(0.5))

model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(), metrics='accuracy')

# Use this cell in implement task 2.3 
# Batch size: refers to the size of data we use in a single pass; 
batch_size = 128

  # Epochs, the number of times to iterate over the dataset  
epochs = 15 

model.fit(X_train,y_train, batch_size=batch_size, epochs = epochs,validation_data=(X_test,y_test))

# Use this cell to implement task 2.4
loss = (loss_score, accuracy_score) = model.evaluate(X_test,y_test,verbose=0)

print("Loss ",loss)

Epoch 1/15
469/469 [==============================] - 2s 4ms/step - loss: 7.7040 - accuracy: 0.1468 - val_loss: 12.1498 - val_accuracy: 0.2457
Epoch 2/15
469/469 [==============================] - 2s 4ms/step - loss: 7.1410 - accuracy: 0.1797 - val_loss: 11.6810 - val_accuracy: 0.2751
Epoch 3/15
469/469 [==============================] - 2s 3ms/step - loss: 7.0358 - accuracy: 0.1839 - val_loss: 12.1519 - val_accuracy: 0.2458
Epoch 4/15
469/469 [==============================] - 2s 5ms/step - loss: 6.9700 - accuracy: 0.1886 - val_loss: 11.6112 - val_accuracy: 0.2796
Epoch 5/15
469/469 [==============================] - 2s 3ms/step - loss: 6.9518 - accuracy: 0.1921 - val_loss: 11.6670 - val_accuracy: 0.2760
Epoch 6/15
469/469 [==============================] - 3s 6ms/step - loss: 6.9411 - accuracy: 0.1902 - val_loss: 11.4801 - val_accuracy: 0.2877
Epoch 7/15
469/469 [==============================] - 3s 5ms/step - loss: 6.8568 - accuracy: 0.1945 - val_loss: 11.4722 - val_accuracy: 0.2879

**Apply the trained model**

Once the model is trained, and the evaluation shows an acceptable performance, we can apply the model to make prediction on new unseen data using the `model.predict` method. The `model.predict(X_test)` returns a ndarry of shape (obs, 10) which represents a probability vector over categories. To make a concrete class assignment we can return the class with the hieghest predicted probability. We can identify the entry with the highest probability for each vector by using the `np.argmax` method accross each row (i.e. accross axis 1). The code above illustrates how to do that.

```python
y_predict = np.argmax(model.predict(X_test),axis=1)
```

**Task 2.5** Apply the model you trained in task 2.4 on the test set and report its performance. Moreover, identify the first three instances the model misclassifies and display their image; indicating in the title the predicted value.


In [41]:
# Use this cell to implement task 2.5 
y_predict = np.argmax(model.predict(X_test),axis=1)
print(y_predict)

[7 2 1 ... 4 5 6]


## Exercise 3
In the first to exercises, you reproduced a network architecture provided to you. In this exercise you are expected to create your own architecture to outperform the model from exercise 2. Repeat all the steps including, defining the network architecture; compile your model, train your model and evaluate your model. You are a allowed to use the `Flatten` layer, and as many `Dense` layers as you like (optionally, we might consider using a `Dropout` layer - which we did not discuss yet). Some things you can try to improve your architecture are
* Introduce mode Dense layers.
* Change the number of nodes in each layer.
* Introduce one or more Dropout layers [see an example tutorial](https://machinelearningmastery.com/how-to-reduce-overfitting-with-dropout-regularization-in-keras/)


In [59]:
# Use this cell to implement task 2.2 
# Implement task 2.1 
import tensorflow.keras 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten, Dropout
from keras.datasets import mnist
import numpy as np
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(196,), name="digits")
x = layers.Flatten()
x = layers.Dense(16, activation="relu", name="dense_1")(inputs)
x = layers.Dense(16, activation="relu", name="dense_2")(x)
model.add(Dropout(.5))

outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

x_val = x_train[-1000:]
y_val = y_train[-1000:]
x_train = x_train[:-1000]
y_train = y_train[:-1000]

model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(), metrics='accuracy')

history = model.fit(x_train, y_train, batch_size=64, epochs=15, validation_data=(x_val, y_val))

print("Evaluate")
results = model.evaluate(x_test, y_test, batch_size=128)
print("loss - acc:", results)
print("shape:", predictions.shape)

Epoch 1/15
922/922 [==============================] - 3s 3ms/step - loss: 2.4869 - accuracy: 0.0998 - val_loss: 14.2162 - val_accuracy: 0.1180
Epoch 2/15
922/922 [==============================] - 3s 3ms/step - loss: 2.4895 - accuracy: 0.1000 - val_loss: 14.2162 - val_accuracy: 0.1180
Epoch 3/15
922/922 [==============================] - 2s 3ms/step - loss: 2.4866 - accuracy: 0.1002 - val_loss: 14.2117 - val_accuracy: 0.1180
Epoch 4/15
922/922 [==============================] - 3s 3ms/step - loss: 2.4815 - accuracy: 0.1004 - val_loss: 14.2045 - val_accuracy: 0.1180
Epoch 5/15
922/922 [==============================] - 2s 3ms/step - loss: 2.4841 - accuracy: 0.1004 - val_loss: 14.2162 - val_accuracy: 0.1180
Epoch 6/15
922/922 [==============================] - 3s 3ms/step - loss: 2.4923 - accuracy: 0.0999 - val_loss: 14.2162 - val_accuracy: 0.1180
Epoch 7/15
922/922 [==============================] - 2s 3ms/step - loss: 2.4941 - accuracy: 0.0997 - val_loss: 14.2162 - val_accuracy: 0.1180

## Compare the models  

In [ ]:
# Use this cell to report the performance of your network, and compare it to the network in exercise 2. 
Exercise 2
accuracy: 0.2087
Loss  [12.752694129943848, 0.2087000012397766]

Exercise 3
accuracy: 0.1262
Loss  [14.070941925048828, 0.12620000541210175]

Copyright Statement: Copyright © 2020 Christoforou. The materials provided by the instructor of this course, including this notebook, are for the use of the students enrolled in the course. Materials are presented in an educational context for personal use and study and should not be shared, distributed, disseminated or sold in print — or digitally — outside the course without permission. You may not, nor may you knowingly allow others to reproduce or distribute lecture notes, course materials as well as any of their derivatives without the instructor's express written consent.